# Exam 2:  CWL Analysis using Spark DataFrames

First let us import the DataFrames that we prepared in the week 9 HW.  Note that if you aren't confident that you did the right thing then please use the solutions that I provided to import them correctly.

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F  # will be used a LOT
from pyspark import Row  # Row will be used in some of the assertions

ss = SparkSession.builder.\
     master('spark://spark-master:7077').\
     appName('cwlanalysis').getOrCreate()

In [2]:
matches_df = ss.read.ggggggg("hdfs://namenode/Users/vagrant/matches_df.parquet")
teammatches_df = ss.read.parquet("hdfs://namenode/Users/vagrant/teammatches_df.parquet")
playermatches_df = ss.read.parquet("hdfs://namenode/Users/vagrant/playermatches_df.parquet")
matchevents_df = ss.read.parquet("hdfs://namenode/Users/vagrant/matchevents_df.parquet")

In [3]:
matches_df.printSchema()

root
 |-- duration_ms: long (nullable = true)
 |-- end_time_s: long (nullable = true)
 |-- hp_hill_rotations: long (nullable = true)
 |-- id: string (nullable = true)
 |-- map: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- rounds: long (nullable = true)
 |-- series_id: string (nullable = true)
 |-- start_time_s: long (nullable = true)
 |-- title: string (nullable = true)



In [4]:
teammatches_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- team: struct (nullable = true)
 |    |-- is_victor: boolean (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- round_scores: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- score: long (nullable = true)
 |    |-- side: string (nullable = true)



In [5]:
playermatches_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- player: struct (nullable = true)
 |    |-- 2piece: long (nullable = true)
 |    |-- 3piece: long (nullable = true)
 |    |-- 4piece: long (nullable = true)
 |    |-- 4streak: long (nullable = true)
 |    |-- 5streak: long (nullable = true)
 |    |-- 6streak: long (nullable = true)
 |    |-- 7streak: long (nullable = true)
 |    |-- 8+streak: long (nullable = true)
 |    |-- accuracy: double (nullable = true)
 |    |-- assists: long (nullable = true)
 |    |-- avg_time_per_life_s: double (nullable = true)
 |    |-- ctf_captures: long (nullable = true)
 |    |-- ctf_defends: long (nullable = true)
 |    |-- ctf_flag_carry_time_s: double (nullable = true)
 |    |-- ctf_kill_carriers: long (nullable = true)
 |    |-- ctf_pickups: long (nullable = true)
 |    |-- ctf_returns: long (nullable = true)
 |    |-- deaths: long (nullable = true)
 |    |-- deaths_per_10min: double (nullable = true)
 |    |-- fave_divisi

In [6]:
matchevents_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- event: struct (nullable = true)
 |    |-- data: struct (nullable = true)
 |    |    |-- attacker: struct (nullable = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- life: long (nullable = true)
 |    |    |    |-- means_of_death: string (nullable = true)
 |    |    |    |-- pos: struct (nullable = true)
 |    |    |    |    |-- x: long (nullable = true)
 |    |    |    |    |-- y: long (nullable = true)
 |    |    |    |-- weapon: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- is_overtime: boolean (nullable = true)
 |    |    |-- life: long (nullable = true)
 |    |    |-- pos: struct (nullable = true)
 |    |    |    |-- x: long (nullable = true)
 |    |    |    |-- y: long (nullable = true)
 |    |    |-- score1: long (nullable = true)
 |    |    |-- score2: long (nullable = true)
 |    |-- round: long (nullable = true)
 |    |-- round_time_m

## Problem 1

Players tend to be very interested in the performance characteristics of various weapons.  One easy question to answer is:

Which weapon was responsible for the most kills over the entire tournament?

Store your result in a `str` variable named `weapon_most_kills`, i.e.
```
weapon_most_kills = 'KICKBOOTY_3000'
```

WE WILL RUN LOW ON MEMORY, so clean up after yourself.  Here is how to delete any DataFrames or RDDs you are done with:
```
del my_df
ss.catalog.clearCache()
```

In [7]:
# YOUR CODE HERE
clean_table = matchevents_df.select('id','event.data.attacker.weapon','event.type')

clean_table.where(clean_table.weapon.isNotNull()).where(clean_table.type == 'death').\
            groupBy('weapon').count().orderBy(['count'],ascending=[0]).show()
            
weapon_most_kills = 'PPSh-41'

+-----------------+-----+
|           weapon|count|
+-----------------+-----+
|          PPSh-41|23422|
|           STG-44|11114|
|              BAR| 3571|
|         No 74 ST| 2117|
|      Springfield|  317|
|   Machine Pistol|  313|
|       Glide Bomb|  312|
|    Fighter Pilot|  226|
|          unknown|  197|
|Artillery Barrage|   97|
|            M1911|   65|
| Mk2 Frag Grenade|   20|
| Stielhandgranate|   12|
|    British No 69|   11|
| M1897 Trench Gun|    2|
|            Luger|    1|
+-----------------+-----+



In [8]:
assert isinstance(weapon_most_kills, str)


## Problem 2:  Winning-est Team

Please rank the teams in order of who won the most *matches* (winning-est team is first).  If there are any ties then secondarily sort alphabetically.

Note that because each team-vs-team competition is the best out of 3 matches, the "winning-est" team might not be the same team that won the whole tournament.

Store your result in a variable named `winningest_teams` that is a `list` of `tuple`s where each tuple contains the team name and the total number of matches won, e.g.
```
[('AWESOME_TEAM', 55), ('OK_TEAM', 43), ...]
```

In [9]:
subset_table = teammatches_df.select('team.name','team.is_victor')
clean_table = subset_table.select([F.col('name'),F.col('is_victor').cast("integer")])
summed_table = clean_table.groupBy('name').sum('is_victor')
summed_table = summed_table.orderBy(["sum(is_victor)", "name"], ascending=[0, 1])
df = summed_table.select('name',summed_table['sum(is_victor)'].alias('win_times'))

key = list(df.select('name').toPandas()['name'])
value = list(df.select('win_times').toPandas()['win_times'])
d = dict(zip(key,value))
winningest_teams = list(d.items())
winningest_teams

[('FAZE CLAN', 28),
 ('EVIL GENIUSES', 24),
 ('TEAM KALIBER', 24),
 ('LUMINOSITY', 19),
 ('EUNITED', 18),
 ('RED RESERVE', 16),
 ('ELEVATE', 13),
 ('LIGHTNING PANDAS', 12),
 ('RISE NATION', 12),
 ('UNILAD', 12),
 ('GHOST GAMING', 11),
 ('TEAM SWEEN', 11),
 ('LETHAL GAMING', 10),
 ('TEAM ENVY', 10),
 ('TEAM VITALITY', 9),
 ('COMPLEXITY', 8),
 ('HERETICS', 7),
 ('MINDFREAK', 6),
 ('OPTIC GAMING', 6),
 ('TEAM ENVYUS', 6),
 ('BRASH', 5),
 ('ENIGMA6', 5),
 ('SPLYCE', 5),
 ('TAINTED MINDS', 5),
 ('ECHO FOX', 4),
 ('MENTALITY', 3),
 ('EPSILON', 2),
 ('MORITURI ESPORTS', 2),
 ('SUPREMACY', 1),
 ('TEAM PRISMATIC', 1),
 ('ZONEGG', 1),
 ('EZG', 0),
 ('HAVOK', 0)]

In [10]:
assert isinstance(winningest_teams, list)
assert isinstance(winningest_teams[0], tuple)
assert isinstance(winningest_teams[0][0], str)
assert isinstance(winningest_teams[0][1], int)


## Problem 3:  Deadliest map

It is interesting to know which maps are "deadliest" (i.e. have the most kills over the entire tournament) because this is where the exciting action was happening.

Similar to Problem 2, provide a rank-order of maps (deadliest first), i.e. your `list` of `tuples` should look like:
```
[('Super Deadly Map', 1053), ('Second deadliest map', 997), ...]
```
where each pair contains the team name and total number of deaths that occurred in that map for the entire tournament.

If there are any ties then secondarily sort alphabetically.  Name your variable `deadliest_maps`.

In [11]:
map_df = matches_df.select('id','map')

kill_table = matchevents_df.select(matchevents_df['id'].alias('kill_id'),'event.type')
kill_table = kill_table.where(kill_table.type == 'death')

joined_table = kill_table.join(map_df,kill_table.kill_id == map_df.id)
clean_table = joined_table.select('type','map')
counted_table = clean_table.groupBy('map').count()
counted_table = counted_table.select('map',counted_table['count'].alias('kill_times'))

df = counted_table.orderBy(['kill_times','map'],ascending=[0, 1])

key = list(df.select('map').toPandas()['map'])
value = list(df.select('kill_times').toPandas()['kill_times'])
d = dict(zip(key,value))
deadliest_maps = list(d.items())
deadliest_maps

[('London Docks', 14285),
 ('Ardennes Forest', 11331),
 ('Sainte Marie du Mont', 5211),
 ('Flak Tower', 3942),
 ('Valkyrie', 3717),
 ('Gibraltar', 2949),
 ('USS Texas', 362)]

In [12]:
assert isinstance(deadliest_maps, list)
assert isinstance(deadliest_maps[0], tuple)
assert isinstance(deadliest_maps[0][0], str)
assert isinstance(deadliest_maps[0][1], int)


## Problem 4:  Time spent per map

Let's figure out how much time was spent (for the entire tournament) on each map.

Produce a DataFrame named `map_durations_df` that contains two columns:  `map` (the name of the map) and `tot_duration_s` (total *seconds* played on the map for the entire tournament over all matches).

It should be sorted in descending order with the longest played map first.

Hint: I used a UDF to solve this

In [13]:
df = matches_df.select('map','duration_ms')

from pyspark.sql.types import DoubleType

def duration_time_s(ms):
    return ms/1000

duration_udf = F.udf(duration_time_s,DoubleType())

df = df.withColumn('duration_time_s',duration_udf(F.col('duration_ms')))

#----------------------------
df = df.select('map','duration_time_s').groupBy('map').sum('duration_time_s')
df = df.select('map',df['sum(duration_time_s)'].alias('tot_duration_s'))


map_durations_df = df.select('map','tot_duration_s').orderBy(['tot_duration_s','map'],ascending=[0, 1])
map_durations_df.show()

+--------------------+--------------+
|                 map|tot_duration_s|
+--------------------+--------------+
|        London Docks|       52300.0|
|     Ardennes Forest|       48356.0|
|Sainte Marie du Mont|       30781.0|
|          Flak Tower|       16635.0|
|            Valkyrie|       14791.0|
|           Gibraltar|       10263.0|
|           USS Texas|        4166.0|
+--------------------+--------------+



In [14]:
from pyspark.sql import DataFrame
assert isinstance(map_durations_df, DataFrame)
assert map_durations_df.columns == ['map', 'tot_duration_s']


## Problem 5: Deadliest map per unit time

The analysis in Problem 3 is not really fair.  Since some maps were played longer than others, we should really produce a DataFrame that details the deadliest maps PER SECOND.

Your resulting DataFrame should be named `deadliest_maps_per_second_df` and have the columns `map` (the map name) and `deaths_per_second` (which will be doubles).

Your DataFrame should be sorted according to `deaths_per_second` (deadliest first) and, in case of tie, alphabetically.

HINT: I used a UDF to solve this

In [19]:
#-----------------------
map_df = matches_df.select('id','map')

kill_table = matchevents_df.select(matchevents_df['id'].alias('kill_id'),'event.type')
kill_table = kill_table.where(kill_table.type == 'death')

joined_table = kill_table.join(map_df,kill_table.kill_id == map_df.id)
clean_table = joined_table.select('type','map')
counted_table = clean_table.groupBy('map').count()
counted_table = counted_table.select('map',counted_table['count'].alias('kill_times'))

deadtimes_df = counted_table.orderBy(['kill_times','map'],ascending=[0, 1])

#-----------------------
df = matches_df.select('map','duration_ms')

from pyspark.sql.types import DoubleType

def duration_time_s(ms):
    return ms/1000

duration_udf = F.udf(duration_time_s,DoubleType())

df = df.withColumn('duration_time_s',duration_udf(F.col('duration_ms')))

df = df.select('map','duration_time_s').groupBy('map').sum('duration_time_s')
df = df.select('map',df['sum(duration_time_s)'].alias('tot_duration_s'))

map_durations_df = df.select('map','tot_duration_s').orderBy(['tot_duration_s','map'],ascending=[0, 1])
map_durations_df = map_durations_df.select('tot_duration_s',map_durations_df['map'].alias('mapmap'))

#-----------------------
joined_table = deadtimes_df.join(map_durations_df,deadtimes_df.map == map_durations_df.mapmap)
joined_table = joined_table.select('map','kill_times','tot_duration_s')

#-----------------------
from pyspark.sql.types import DoubleType

def per_second(kill_times,tot_dur):
    return kill_times/tot_dur

per_second_udf = F.udf(per_second,DoubleType())

deadliest_maps_per_second_df = joined_table.withColumn('deaths_per_second', \
                                                       per_second_udf(F.col('kill_times'),\
                                                                      F.col('tot_duration_s')))

#-----------------------
deadliest_maps_per_second_df = deadliest_maps_per_second_df.orderBy(['deaths_per_second','map'],ascending = [0,1])
deadliest_maps_per_second_df = deadliest_maps_per_second_df.select('map','deaths_per_second')
deadliest_maps_per_second_df.show()



+--------------------+-------------------+
|                 map|  deaths_per_second|
+--------------------+-------------------+
|           Gibraltar|0.28734288219818765|
|        London Docks| 0.2731357552581262|
|            Valkyrie| 0.2513014671083767|
|          Flak Tower| 0.2369702434625789|
|     Ardennes Forest| 0.2343245926048474|
|Sainte Marie du Mont|0.16929274552483675|
|           USS Texas|0.08689390302448392|
+--------------------+-------------------+



In [16]:
import numpy as np

assert isinstance(deadliest_maps_per_second_df, DataFrame)
assert deadliest_maps_per_second_df.columns == ['map', 'deaths_per_second']
assert deadliest_maps_per_second_df.select('deaths_per_second').dtypes[0][1] == 'double'


## Problem 6: Cumulative time a team played

Create a DataFrame that contains the matches played by the team `EVIL GENIUSES` (one row per match).  This DataFrame, named `genius_matches`, will contain 4 columns:

`mode`, `start_time_s`, `end_time_s`, and `cumulative_time_s` where `cumulative_time_s` is the cumulative seconds played UP TO AND INCLUDING THAT MATCH.  The cumulative match time should be separate for each game mode.

The matches should be sorted first by `mode`, then in order of time played (earliest matches first).

HINT:  You need some UDFs and Windowing

In [17]:
team_table = teammatches_df.select('team.name','id','mode')
team_table = team_table.where(team_table.name == 'EVIL GENIUSES')
match_table = matches_df.select('id','start_time_s','end_time_s','duration_ms')
match_table = match_table.select('start_time_s','end_time_s','duration_ms',match_table['id'].alias('idid'))

joined_table = team_table.join(match_table,team_table.id == match_table.idid)
joined_table = joined_table.select('id','mode','name','start_time_s','end_time_s','duration_ms')

joined_table = joined_table.sort('start_time_s')

#----------------------------------------
from pyspark.sql.types import DoubleType

def duration_time_s(ms):
    return ms/1000

duration_udf = F.udf(duration_time_s,DoubleType())

udf_df = joined_table.withColumn('duration_time_s',duration_udf(F.col('duration_ms')))


#----------------------------
from pyspark.sql import Window

window = Window.partitionBy('mode').\
         orderBy('start_time_s').\
         rowsBetween(Window.unboundedPreceding,0)

df = udf_df.withColumn('cumulative_time_s',F.sum(F.col('duration_time_s')).over(window))


#----------------------------
genius_matches = df.select('mode','start_time_s','end_time_s','cumulative_time_s')
genius_matches.show(20)


+----------------+------------+----------+-----------------+
|            mode|start_time_s|end_time_s|cumulative_time_s|
+----------------+------------+----------+-----------------+
|       Hardpoint|  1534361982|1534362687|            705.0|
|       Hardpoint|  1534364818|1534365529|           1416.0|
|       Hardpoint|  1534440741|1534441439|           2114.0|
|       Hardpoint|  1534443800|1534444566|           2880.0|
|       Hardpoint|  1534457556|1534458220|           3544.0|
|       Hardpoint|  1534528542|1534529055|           4057.0|
|       Hardpoint|  1534608284|1534609063|           4836.0|
|       Hardpoint|  1534625528|1534626100|           5408.0|
|       Hardpoint|  1534628787|1534629444|           6065.0|
|       Hardpoint|  1534700869|1534701580|           6776.0|
|       Hardpoint|  1534704083|1534704675|           7368.0|
|       Hardpoint|  1534712942|1534713615|           8041.0|
|       Hardpoint|  1534715919|1534716637|           8759.0|
|       Hardpoint|  1534

In [18]:
assert isinstance(genius_matches, DataFrame)
assert genius_matches.columns == \
    ['mode', 'start_time_s', 'end_time_s', 'cumulative_time_s']
